In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import seaborn as sns
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error

In [ ]:
data=pd.read_excel("jharkhand.xlsx")

In [ ]:
data['Timestamp']=pd.to_datetime(data.Timestamp)

In [ ]:
data2=data

In [ ]:
percentile25 = data['demand'].quantile(0.05)
percentile75 = data['demand'].quantile(0.95)

In [ ]:
iqr = percentile75 - percentile25
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr

In [ ]:
new_df = data.copy()

new_df['demand'] = np.where(
    new_df['demand'] > upper_limit,
    upper_limit,
    np.where(
        new_df['demand'] < lower_limit,
        lower_limit,
        new_df['demand']
    )
)

In [ ]:
df=new_df.iloc[:]

In [ ]:
lis = df['demand'].tolist()
lis2=data['demand'].tolist()

In [ ]:
def create_1(lis):
    creating_new=[]
    for i in range(43968):
        if(i<96):
            creating_new.append(lis[i])
        else:
            creating_new.append(lis[i-96])
    return creating_new

In [ ]:
def create_2(lis):
    creating_new_2=[]
    for i in range(43968):
        if(i<96*7):
            creating_new_2.append(lis[i])
        else:
            creating_new_2.append(lis[i-96*7])
    return creating_new_2

In [ ]:
df['demand_prev']=create_1(lis)
df['demand_prev_week']=create_2(lis)
data['demand_prev']=create_1(lis2)
data['demand_prev_week']=create_2(lis2)

In [ ]:
cols=list(df)[1:4]

In [ ]:
df1= df.iloc[:38400]
df2=data.iloc[38400:]

In [ ]:
dataset=df[cols].astype(float)
train_1 = df1[cols].astype(float)
test_1 = df2[cols].astype(float)

In [ ]:
# scaler=MinMaxScaler(feature_range=(0,1))
# dataset=scaler.fit_transform(dataset)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler = scaler.fit(train_1)
train = scaler.transform(train_1)
test = scaler.transform(test_1)

In [ ]:
# Using 400 days as train and 58 days for testing

In [ ]:
n_future=1
n_past=96

In [ ]:
def splitter(dataset, n_past,n_future):
    X=[]
    Y=[]
    for i in range(n_past,len(dataset) - n_future+1):
        X.append(dataset[i-n_past:i,0:dataset.shape[1]])
        Y.append(dataset[i+n_future-1:i+n_future,0])
    return np.array(X),np.array(Y)


In [ ]:
trainX,trainY=splitter(train,n_past,n_future)
testX,testY=splitter(test,n_past,n_future)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
history = model.fit(trainX, trainY, epochs=2, batch_size=16, validation_split=0.1, verbose=1)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
prediction_copies = np.repeat(trainPredict, train.shape[1], axis=-1)
trainPredict = scaler.inverse_transform(prediction_copies)[:,0]

In [ ]:
trainPredict

In [ ]:
prediction_copies = np.repeat(testPredict, train.shape[1], axis=-1)
testPredict = scaler.inverse_transform(prediction_copies)[:,0]

In [ ]:
dfnew=df.iloc[38400+96:]
dfnew['predict'] = testPredict

In [ ]:
dataset=data['demand'].astype(float)

In [ ]:
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:] = np.nan
trainPredictPlot[96:len(trainPredict)+96] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:] = np.nan
testPredictPlot[len(trainPredict)+(96*2)-1:len(dataset)-1] = testPredict

# plot baseline and predictions
plt.figure(figsize=(16, 8))
plt.plot(dataset)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
dataset2=data2['demand'].astype(float)

In [ ]:
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:] = np.nan
trainPredictPlot[96:len(trainPredict)+96] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:] = np.nan
testPredictPlot[len(trainPredict)+(96*2)-1:len(dataset)-1] = testPredict

# plot baseline and predictions
plt.figure(figsize=(16, 8))
plt.plot(dataset2)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
def mean_squared_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.square((y_true - y_pred) / y_true)) * 100

In [ ]:
mspe = mean_squared_percentage_error(dfnew['demand'],testPredict[:])
print(f'Mean Squared Percentage Error: {mspe:.2f}%')